# Query CELLxGENE

We want to retrieve the census general information and single-cell metadata to extract useful statistics.


## Environment setup

We need to import the packages and functions that we are going to use.


In [1]:
import cellxgene_census
import os
import pyarrow.dataset as ds
import session_info
import sys
import tiledbsoma

from datetime import date

In [2]:
# Make sure the module can be found
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the function that will combine the parquet files from utils.py
from src.utils import combine_parquet

We also want to setup the output directory.


In [3]:
## Force execution
force_execution = False
## Set census release into format YYYY-MM-DD
census_version = "2023-12-15"
## Set output directory
output_dir = "../data/" + census_version + "/"
## Create output directory
if force_execution or (not os.path.exists(path=output_dir)):
    os.makedirs(name=output_dir, exist_ok=True)
## Set subfolder to store chunks
chunk_dir = output_dir + "sc_metadata_chunks/"
## Create the directory
if force_execution or (not os.path.exists(path=chunk_dir)):
    os.makedirs(name=chunk_dir, exist_ok=True)

## Querying and reading Census metadata


### 1. Open a connection

The first step is to open a connection. We can do it by using the `?open_soma` function from the [cellxgene.census](https://github.com/chanzuckerberg/cellxgene-census/tree/main/api/r/cellxgene.census) R package.


In [4]:
## Open a specific version of the Census
census = cellxgene_census.open_soma(census_version=census_version)

### 2. Retrieve the information

At this point, we can use TileDB-SOMA operations to retrieve the information we need.

We are mainly interested in two things:

1. The information from the `"census_info"` item. It contains tables as `SOMADataFrame` providing information of the census as a whole.
2. The cell metadata, contained in the `"obs"` `SOMADataFrame` of the `"census_data"` item.


#### 2.1 Get Census summary information

First, let's retrieve the Census summary information:


In [5]:
census_info = census["census_info"]

Then, we retrieve the three tables containing the information on the summary as a whole:

- `"summary"`: high-level information of this Census, e.g. build date, total cell count, etc.
- `"datasets"`: A table with all datasets from CELLxGENE Discover used to create the Census.
- `"summary_cell_counts"`: Cell counts stratified by relevant cell metadata.


In [6]:
# Get the summary
census_info_summary = census_info["summary"]  # Returns a SOMADataFrame
# Get the summary
census_info_datasets = census_info["datasets"]  # Returns a SOMADataFrame
# Get the summary
census_info_summary_cell_counts = census_info[
    "summary_cell_counts"
]  # Returns a SOMADataFrame

All read operations must be performed using the `.read()` method which returns a [TableReadIter]().


In [7]:
# Read as an iterator of Arrow Tables
census_info_summary = census_info_summary.read()
census_info_datasets = census_info_datasets.read()
census_info_summary_cell_counts = census_info_summary_cell_counts.read()

# Print
census_info_summary

We can use `.concat()` to retrieve all metadata at once and obtain a [PyArrow.Table](https://arrow.apache.org/docs/python/generated/pyarrow.Table.html).


In [8]:
# Retrieve all metadata at once
census_info_summary = census_info_summary.concat()
census_info_datasets = census_info_datasets.concat()
census_info_summary_cell_counts = census_info_summary_cell_counts.concat()

# Print
census_info_summary

pyarrow.Table
soma_joinid: int64
label: large_string
value: large_string
----
soma_joinid: [[0,1,2,3,4,5,6]]
label: [["census_schema_version","census_build_date","dataset_schema_version","total_cell_count","unique_cell_count","number_donors_homo_sapiens","number_donors_mus_musculus"]]
value: [["1.2.0","2023-10-23","3.1.0","68683222","40356133","15588","1990"]]

We can convert the `DataFrame` to a different format, like a [Pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).


In [34]:
# Transform to Pandas DataFrame
census_info_summary = census_info_summary.to_pandas()
census_info_datasets = census_info_datasets.to_pandas()
census_info_summary_cell_counts = census_info_summary_cell_counts.to_pandas()

In [21]:
census_info_summary

,soma_joinid,label,value
0,0,census_schema_version,1.2.0
1,1,census_build_date,2023-10-23
2,2,dataset_schema_version,3.1.0
3,3,total_cell_count,68683222
4,4,unique_cell_count,40356133
5,5,number_donors_homo_sapiens,15588
6,6,number_donors_mus_musculus,1990


In [22]:
census_info_datasets

,soma_joinid,collection_id,collection_name,collection_doi,dataset_id,dataset_version_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
0,0,4dca242c-d302-4dba-a68f-4c61e7bad553,Comparative transcriptomics reveals human-spec...,10.1126/science.ade9516,2bdd3a2c-2ff4-4314-adf3-8a06b797a33a,7eb7f2fd-fd74-4c99-863c-97836415652e,Human: Great apes study,2bdd3a2c-2ff4-4314-adf3-8a06b797a33a.h5ad,156285
1,1,d17249d2-0e6e-4500-abb8-e6c93fa1ac6f,Transcriptomic cytoarchitecture reveals princi...,10.1126/science.adf6812,f5b0810c-1664-4a62-ad06-be1d9964aa8b,d4427196-7876-4bdd-a929-ae4d177ec776,Dissection: Angular gyrus (AnG),f5b0810c-1664-4a62-ad06-be1d9964aa8b.h5ad,110752
2,2,d17249d2-0e6e-4500-abb8-e6c93fa1ac6f,Transcriptomic cytoarchitecture reveals princi...,10.1126/science.adf6812,e4ddac12-f48f-4455-8e8d-c2a48a683437,3280113b-7148-4a3e-98d4-015f443aab8a,Supercluster: CGE-derived interneurons,e4ddac12-f48f-4455-8e8d-c2a48a683437.h5ad,129495
3,3,d17249d2-0e6e-4500-abb8-e6c93fa1ac6f,Transcriptomic cytoarchitecture reveals princi...,10.1126/science.adf6812,e2808a6e-e2ea-41b9-b38c-4a08f1677f02,dc092185-3b8e-4fcb-ae21-1dc106d683ac,Dissection: Primary auditory cortex(A1),e2808a6e-e2ea-41b9-b38c-4a08f1677f02.h5ad,139054
4,4,d17249d2-0e6e-4500-abb8-e6c93fa1ac6f,Transcriptomic cytoarchitecture reveals princi...,10.1126/science.adf6812,d01c9dff-abd1-4825-bf30-2eb2ba74597e,c4959ded-83dc-4442-aac7-9a59bdb47801,Supercluster: Deep layer (non-IT) excitatory n...,d01c9dff-abd1-4825-bf30-2eb2ba74597e.h5ad,92969
...,...,...,...,...,...,...,...,...,...
646,646,180bff9c-c8a5-4539-b13b-ddbc00d643e6,Molecular characterization of selectively vuln...,10.1038/s41593-020-00764-7,f9ad5649-f372-43e1-a3a8-423383e5a8a2,0912e658-ccd7-43e9-8d81-4349432115f9,Molecular characterization of selectively vuln...,f9ad5649-f372-43e1-a3a8-423383e5a8a2.h5ad,8168
647,647,a72afd53-ab92-4511-88da-252fb0e26b9a,Single-cell atlas of peripheral immune respons...,10.1038/s41591-020-0944-y,456e8b9b-f872-488b-871d-94534090a865,fcc85817-ef31-4056-9a96-b3730ccec522,Single-cell atlas of peripheral immune respons...,456e8b9b-f872-488b-871d-94534090a865.h5ad,44721
648,648,38833785-fac5-48fd-944a-0f62a4c23ed1,Construction of a human cell landscape at sing...,10.1038/s41586-020-2157-4,2adb1f8a-a6b1-4909-8ee8-484814e2d4bf,998d8dbd-2f42-4611-9973-2da95db46c29,Construction of a human cell landscape at sing...,2adb1f8a-a6b1-4909-8ee8-484814e2d4bf.h5ad,598266
649,649,5d445965-6f1a-4b68-ba3a-b8f765155d3a,A molecular cell atlas of the human lung from ...,10.1038/s41586-020-2922-4,e04daea4-4412-45b5-989e-76a9be070a89,d54c8b65-3e6c-4265-98aa-5f082a5f3816,"Krasnow Lab Human Lung Cell Atlas, Smart-seq2",e04daea4-4412-45b5-989e-76a9be070a89.h5ad,9409


In [23]:
census_info_summary_cell_counts

,soma_joinid,organism,category,ontology_term_id,unique_cell_count,total_cell_count,label
0,0,Homo sapiens,all,na,36227903,62998417,na
1,1,Homo sapiens,assay,EFO:0008722,264166,279635,Drop-seq
2,2,Homo sapiens,assay,EFO:0008780,25652,51304,inDrop
3,3,Homo sapiens,assay,EFO:0008796,54753,54753,MARS-seq
4,4,Homo sapiens,assay,EFO:0008919,89477,206754,Seq-Well
...,...,...,...,...,...,...,...
1406,1406,Mus musculus,tissue_general,UBERON:0002113,179684,231804,kidney
1407,1407,Mus musculus,tissue_general,UBERON:0002365,15577,46731,exocrine gland
1408,1408,Mus musculus,tissue_general,UBERON:0002367,37715,130135,prostate gland
1409,1409,Mus musculus,tissue_general,UBERON:0002368,13322,39966,endocrine gland


Finally, we store the data for future usage.


In [35]:
## Save the data
census_info_summary.to_csv(output_dir + "census_info_summary.csv", index=False)
census_info_datasets.to_csv(output_dir + "census_info_datasets.csv", index=False)
census_info_summary_cell_counts.to_csv(
    output_dir + "census_info_summary_cell_counts.csv", index=False
)

#### 2.2 Get Census summary information

Now we can retrieve the Census single-cell information.


In [9]:
## Get the census data
census_data = census["census_data"]

Then, we want to select the species:


In [10]:
## Get census single-cell info for homo sapiens
census_hsapiens_data = census_data["homo_sapiens"]

Now, we can select the cell metadata:


In [11]:
## Get census single-cell metadata info for homo sapiens
census_hsapiens_cell_metadata = census_hsapiens_data["obs"]

## Print
census_hsapiens_cell_metadata

<DataFrame 's3://cellxgene-census-public-us-west-2/cell-census/2023-12-15/soma/census_data/homo_sapiens/obs' (open for 'r')>

Reading the data as we previously did could raise issues related to memory. To avoid this type of problem, we can read the data by chunk.


In [12]:
## Read as an iterator of Arrow Tables
census_hsapiens_cell_metadata = census_hsapiens_cell_metadata.read()

## Print
census_hsapiens_cell_metadata

As we can see, the `read()` function returned a `TableReadIter` object. Looking at the [documentation](https://github.com/single-cell-data/TileDB-SOMA/blob/main/apis/python/src/tiledbsoma/_read_iters.py), we can see that it contains an `sr` element which is a `SOMAArray` object. We can use this to retrieve data in chunks.


In [38]:
## Get SOMAArray
census_hsapiens_cell_metadata_somaarray = census_hsapiens_cell_metadata.sr

## Print
census_hsapiens_cell_metadata_somaarray

Now we can retrieve the data in chunks using the `read_next()` function.


In [39]:
## Init index variable
i = 1
## Read next chunk of data in memory
census_hsapiens_cell_metadata_chunk = (
    census_hsapiens_cell_metadata_somaarray.read_next()
)
## Continue until complete
while census_hsapiens_cell_metadata_chunk is not None:
    ## Convert to Pandas
    census_hsapiens_cell_metadata_chunk = (
        census_hsapiens_cell_metadata_chunk.to_pandas()
    )
    ## File name
    fn = "hsapiens_cell_metadata_chunk_" + str(i)
    ## File path
    fp = chunk_dir + fn + ".parquet"
    ## Save as parquet file
    census_hsapiens_cell_metadata_chunk.to_parquet(path=fp, index=False)
    ## Print log
    print("Saved chunk " + str(i) + " to " + fp)
    ## Update
    i = i + 1
    ## Read next chunk of data in memory
    census_hsapiens_cell_metadata_chunk = (
        census_hsapiens_cell_metadata_somaarray.read_next()
    )

We can merge the chunks in a single `.parquet` file for future usage. First, we can retrieve the paths to the chunk files using the [`pyarrow.dataset.dataset()`](https://arrow.apache.org/docs/python/generated/pyarrow.dataset.dataset.html#pyarrow-dataset-dataset) function:


In [5]:
## Retrieve all parquet files
dataset = ds.dataset(chunk_dir, format="parquet")
## Print
dataset

We can print the file paths to make sure that all the files were found:


In [6]:
dataset.files

['../data/2023-12-15/sc_metadata_chunks/hsapiens_cell_metadata_1.parquet',
 '../data/2023-12-15/sc_metadata_chunks/hsapiens_cell_metadata_2.parquet',
 '../data/2023-12-15/sc_metadata_chunks/hsapiens_cell_metadata_3.parquet']

Now we can combine the files into a unique `.parquet` file.

We could use the common approach of converting the `FileSystemDataset` to a `pyarrow.Table` with `to_table()`, and then converting it to a Pandas object with `to_pandas()`, but these steps would load the data into memory and could create issues, most of all when the file size is big.

We can instead take advantage of a function we wrote that uses `pyarrow.parquet` functionalities to read data in batch and write it to the Parquet file.


In [8]:
## Save the data after loading the files in memory - DO NOT DO THIS
# dataset.to_table().to_pandas().to_parquet(path=fp, index=False) # DO NOT DO THIS

In [8]:
## File path
fp = output_dir + "hsapiens_cell_metadata.parquet"
## Combine all the chunks
combine_parquet(input_files=dataset.files, output_file=fp)

### 3. Close the Census

Finally, we want to close the connection.


In [ ]:
## Close the connection
census.close()

## Session info

The version number of Python and packages loaded for generating the analysis.


In [3]:
## show session info
session_info.show()